Dado o desafio de implementar uma rede simples (MLP) usando o Keras, será utilizado um dataset que apresenta informções sobre mulheres diagnosticadas com câncer de mama. Entre as informações estão o tipo de câncer e se houve reincidência ou não. O objetivo do modelo é prever gravidade do câncer, isso acontece através da coluna output_os, se for 0 não é grave e se for 1 é.

# Importando as bibliotecas

In [26]:
!pip install tensorflow

In [27]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.metrics import Precision, Recall
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Baixando os dados

In [3]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/DataSet3.csv')
df.head()

,Unnamed: 0,dados_antropometricos,dados_histopatologicos_mama,registro_de_tumores,ki67_perc_mean,ki67_perc_last,height,weight,follow_up_days_recidive,follow_up_days_recidive_mean,follow_up_days_recidive_std,sister_n,bmi_mean,bmi_std,record_id,output_os
0,0,20.0,3.0,3.0,15.0,15.0,152.100000,67.335000,1338.0,1338.0,0.0,-1.0,29.114000,0.435775,54,1
1,1,25.0,1.0,1.0,-1.0,-1.0,157.909091,59.916000,-1.0,-1.0,0.0,-1.0,24.403571,1.296000,302,1
2,2,35.0,1.0,1.0,20.0,20.0,155.117647,63.451471,2442.0,2442.0,0.0,-1.0,25.668421,4.313029,710,1
3,3,10.0,1.0,1.0,-1.0,-1.0,152.000000,83.110000,-1.0,-1.0,0.0,-1.0,35.961538,0.525259,752,1
4,4,19.0,1.0,1.0,-1.0,-1.0,167.000000,64.173684,-1.0,-1.0,0.0,-1.0,23.040909,0.518844,1589,1


# Análise Exploratória

In [4]:
df.output_os.unique()

array([1, 0])

In [5]:
df.follow_up_days_recidive.value_counts()

,count
follow_up_days_recidive,
-1.0,2862
276.0,6
309.0,5
251.0,5
155.0,5
...,...
636.0,1
2375.0,1
379.0,1


# Separando os dados

In [6]:
from sklearn.model_selection import train_test_split

In [7]:
X = df.drop(["record_id", "output_os"], axis=1)
y = df.output_os

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=3)

In [29]:
model = Sequential()
model.add(Dense(1, activation='sigmoid', input_shape=(X_train.shape[1],)))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [30]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Modelagem

In [31]:
class F1Metric(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        if 'accuracy' in logs:
            y_pred = (self.model.predict(X_test) > 0.5).astype("int32")
            f1 = f1_score(y_test, y_pred)
            print(f"Epoch {epoch+1} - F1 Score: {f1}")

In [32]:
model.fit(X_train, y_train, epochs=50, batch_size=10, callbacks=[F1Metric()])

Epoch 1/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
Epoch 1 - F1 Score: 0.5980126467931346
331/331 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.4722 - loss: 568.6448
Epoch 2/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
Epoch 2 - F1 Score: 0.3076923076923077
331/331 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.4822 - loss: 199.7442
Epoch 3/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
Epoch 3 - F1 Score: 0.6145695364238409
331/331 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.4493 - loss: 31.5939
Epoch 4/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
Epoch 4 - F1 Score: 0.6674391657010428
331/331 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6376 - loss: 1.5888
Epoch 5/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
Epoch 5 - F1 Score: 0.6524317912218268
331/331 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6242 - loss: 1.1008
Epoch 6/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
Epoch 6 - F1 Score: 0.49498327759197325
331/331 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6009 - loss: 0.8778
Epoch

In [36]:
y_pred_proba = model.predict(X_test)
y_pred = (y_pred_proba > 0.5).astype("int32")

26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step


# Avaliação

* Accuracy
* F1-Score

In [37]:
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"Acurácia: {accuracy}")
print(f"F1 Score: {f1}")

Acurácia: 0.6964933494558646
F1 Score: 0.6367583212735167


A acurácia é a proporção de previsões corretas feitas pelo modelo em relação ao total de previsões. O F1 Score é a média harmônica entre a precisão e o recall,. É especialmente útil quando você precisa de um equilíbrio entre precisão e recall e há um desbalanceamento entre as classes. O presente modelo se diferencia por usar dados reais e podendo ser útil para aconselhamento de tratamento, uma vez que as métricas atingidas são satisfatórias. Uma maneira de deixá-lo ainda melhor é ajustar o Threshold para além do padrão de 0.5 em busca de molheres métricas de desempenho.